# TD3-Portfoliomanagement für den Dow Jones 30

Dieses Notebook zeigt, wie sich mit dem FinRL-Framework und dem TD3-Algorithmus (Twin Delayed DDPG) eine dynamische Portfolio-Allokation für den Dow Jones 30 aufbauen lässt. Alle Abschnitte sind kommentiert, um auch Einsteiger*innen bei jedem Schritt mitzunehmen.


In [47]:
# Paketinstallation (ggf. beim ersten Start ausführen)
# Hinweis (Deutsch): Die Installation kann ein paar Minuten dauern.
# Achtung: FinRL wird direkt aus dem GitHub-Master installiert, da dort macOS-kompatible Abhängigkeiten gepflegt werden.
%pip install -q "git+https://github.com/AI4Finance-Foundation/FinRL.git@master" ta


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Basisimporte und Verzeichnisse vorbereiten (Kommentare auf Deutsch)
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from finrl import config
from finrl.config import INDICATORS


from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.plot import convert_daily_return_to_pyfolio_ts

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

from pyfolio import timeseries






In [59]:
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

## Modellpfade für bereits trainierte Agenten

Damit wir nicht bei jedem Lauf erneut trainieren müssen, protokollieren wir hier die Speicherorte der zuletzt gesicherten Checkpoints je Algorithmus.


In [76]:
DOW_30_TICKER = [
    'MMM', 'AXP','AMGN','AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 
    'KO', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'JNJ', 
    'JPM', 'MCD', 'MRK', 'MSFT','NVDA', 'NKE', 'PG', 'CRM','SHW', 'TRV', 
    'UNH', 'VZ', 'V', 'WMT'
]

In [77]:
# Dow-Jones-Konfiguration und Trainings-/Testzeiträume (Kommentare auf Deutsch)
ticker_list = DOW_30_TICKER


train_start_date = "2008-01-01"
train_end_date = "2020-12-31"

validate_start_date = "2021-01-01"
validate_end_date = "2022-12-31"

trade_start_date = "2022-01-01"
trade_end_date = "2025-11-01"

initial_capital = 1_000_000
transaction_cost_pct = 0.001  # 10 Basispunkte pro Trade
hmax = 100  # maximale Stückzahl pro Order
reward_scaling = 1e-4  
num_stock_shares = 1000


## Schritt 1: Daten- und Marktparameter festlegen

Wir definieren, welche Dow-Jones-30-Titel, welchen Zeitraum und welche Finanzindikatoren unser Agent für das Training und das anschließende Trading nutzen soll.


In [78]:
df_raw = YahooDownloader(
    start_date=train_start_date,
    end_date=trade_end_date,
    ticker_list= DOW_30_TICKER,
).fetch_data()
df_raw.to_csv('dow_30_data.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (134587, 8)


In [79]:
df_raw

df_raw.isnull().values.any()

np.False_

## Schritt 2: Datenbeschaffung & Feature Engineering

Wir nutzen den integrierten `DataProcessor` von FinRL, der Yahoo Finance ansteuert, sämtliche Dow-Jones-Kurse lädt und anschließend technische Indikatoren hinzufügt.


In [80]:
#tech_indicators = ['rsi','macd']

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=False,        
    use_turbulence=False,  
    user_defined_feature=False
)
df = fe.preprocess_data(df_raw)

df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []


lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


df


Successfully added technical indicators


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,2.559735,2.631413,2.559435,2.578329,607541200,AAPL,2,-0.081830,3.064709,2.431330,42.254776,-80.847429,16.129997,2.723836,2.834898,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
1,2008-12-31,39.332115,39.652222,38.862174,38.896227,6287200,AMGN,2,0.147362,40.147627,38.404895,51.060626,51.895814,10.431925,38.597457,38.170424,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
2,2008-12-31,2.564000,2.584500,2.495500,2.537000,155844000,AMZN,2,0.048231,2.712887,2.319763,49.073148,58.457892,4.441536,2.344533,2.466917,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
3,2008-12-31,14.254675,14.408365,13.762870,13.808976,9625600,AXP,2,-0.914660,18.229860,12.377336,42.554853,-74.811285,25.740060,15.391969,17.222408,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
4,2008-12-31,32.005890,32.290921,31.128298,31.195806,5443100,BA,2,-0.279800,32.174385,28.867830,47.440244,157.922054,5.392003,30.327214,32.389914,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122839,2025-10-31,344.135773,346.390513,342.629284,344.375203,1558200,SHW,4,-0.114875,349.007182,323.242812,50.008121,68.312113,40.070235,338.373908,349.850782,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...
122840,2025-10-31,268.619995,271.100006,267.750000,268.320007,1900900,TRV,4,-1.574636,283.656000,258.556009,48.494234,-54.270578,2.088467,273.356336,272.521841,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...
122841,2025-10-31,341.559998,346.279999,337.119995,344.390015,8390800,UNH,4,2.566168,374.210235,345.512769,50.206397,-128.399797,6.756073,356.012333,332.561928,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...
122842,2025-10-31,39.740002,39.959999,38.779999,38.980000,52181800,VZ,4,-0.708339,41.363981,38.558197,43.848153,-77.341672,12.228777,40.887493,42.056723,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...


In [81]:
train = data_split(df, start=train_start_date, end=train_end_date)
validate = data_split(df, start=validate_start_date, end=validate_end_date)
trade = data_split(df, start=trade_start_date, end=trade_end_date)

## Schritt 3: Handelsumgebung definieren

Wir erstellen zwei `StockTradingEnv`-Instanzen – eine für das Training und eine für das spätere Trading/Backtesting. Die Umgebung erhält Angaben zu Kapital, Transaktionskosten, Risiko-Kontrollen und zu den technischen Indikatoren.


In [82]:

stock_dim = len(train.tic.unique())

print(f"State-Dimension: {stock_dim} | Aktienanzahl: {stock_dim}")

env_kwargs = {
    "hmax": hmax,
    "initial_amount": initial_capital,
    "transaction_cost_pct": transaction_cost_pct,
    "state_space": stock_dim,
    "stock_dim": stock_dim,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dim,
    "reward_scaling": reward_scaling,
}

e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)
e_validate_gym = StockPortfolioEnv(df=validate, **env_kwargs)
e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

State-Dimension: 29 | Aktienanzahl: 29


## Agenten konfigurieren


In [83]:
agent = DRLAgent(env=e_train_gym)


PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 64,
}
model_ppo = agent.get_model("ppo", model_kwargs=PPO_PARAMS)
trained_ppo = agent.train_model(model=model_ppo, 
                                tb_log_name="ppo", 
                                total_timesteps=50000
                                )
trained_ppo.save("trained_models/ppo_portfolio")

DDPG_PARAMS = {
    "batch_size": 128, 
    "buffer_size": 50000, 
    "learning_rate": 0.001
}


model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)
trained_ddpg = agent.train_model(model=model_ddpg, 
                                 tb_log_name="ddpg", 
                                 total_timesteps=30000
                                 )
trained_ddpg.save("trained_models/ddpg_portfolio")

TD3_PARAMS = {
    "batch_size": 100, 
    "buffer_size": 1000000, 
    "learning_rate": 0.001
}
model_td3 = agent.get_model("td3", model_kwargs=TD3_PARAMS)
trained_td3 = agent.train_model(model=model_td3, 
                                tb_log_name="td3", 
                                total_timesteps=30000
                                )
trained_td3.save("trained_models/td3_portfolio")

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| time/              |           |
|    fps             | 1435      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 4968913.0 |
|    reward_max      | 4968913.0 |
|    reward_mean     | 2634738.8 |
|    reward_min      | 806211.2  |
----------------------------------
begin_total_asset:1000000
end_total_asset:10025155.64348428
Sharpe:  1.136653997516722
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 3.02e+03   |
|    ep_rew_mean          | 1.22e+10   |
| time/                   |            |
|    fps                  | 1109       |
|    iterations           | 2          |
|    time_elapsed         | 3   

KeyboardInterrupt: 

In [75]:
# Modellpfade registrieren, damit keine erneuten Trainings nötig sind (Kommentare auf Deutsch)
MODEL_PATHS = {
    "ppo": Path("trained_models/ppo_portfolio.zip"),
    "ddpg": Path("trained_models/ddpg_portfolio.zip"),
    "td3": Path("trained_models/td3_portfolio.zip"),
}


## Baselines Trade Fenster


In [ ]:
baseline_df = get_baseline(
        ticker="^DJI", 
        start = trade_start_date,
        end =  trade_end_date)

baseline_df_stats = backtest_stats(baseline_df, value_col_name = 'close')
baseline_returns = get_daily_return(baseline_df, value_col_name="close")


dji_cumpod =(baseline_returns+1).cumprod()-1

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (962, 8)
Annual return          0.071158
Cumulative returns     0.300063
Annual volatility      0.153786
Sharpe ratio           0.524238
Calmar ratio           0.324317
Stability              0.820039
Max drawdown          -0.219408
Omega ratio            1.097346
Sortino ratio          0.757259
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.959405
Daily value at risk   -0.019055
dtype: float64


## Agenten Trade Fenster

Lade TD3-Modell aus trained_models/td3_portfolio.zip
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
begin_total_asset:1000000
end_total_asset:1510125.9302805392
Sharpe:  0.7428500704902061
hit end!


KeyError: 'account_value'

## Markowitz-Portfolio mit rollierendem Mean-Variance-Ansatz



In [ ]:

# Markowitz Mean-Variance mit rollierendem Fenster und quartalsweisem Rebalancing (Deutsch kommentiert)
MARKOWITZ_LOOKBACK = 252  # 1 Jahr historische Daten für Schätzung
MARKOWITZ_TXN_COST = 0.001  # 0,1% Transaktionskosten
MARKOWITZ_REBAL_FREQ = "QS"  # Quartalsanfang (Quarter Start)

# Preisdaten für gesamten Zeitraum (inkl. Historie) vorbereiten
prices_all = (
    df.pivot_table(index="date", columns="tic", values="close")
      .sort_index()
      .ffill()  # Forward-Fill für fehlende Werte (trägt letzten bekannten Preis vor)
)
prices_all.index = pd.to_datetime(prices_all.index)
prices_trade = prices_all.loc[trade_start_date:trade_end_date]

# KORREKTUR 1: Nur Spalten entfernen, die ALLE Werte fehlen haben (nicht "any")
# Einzelne NaN sollten durch ffill bereits behoben sein
prices_trade = prices_trade.dropna(axis=1, how="all")


def run_markowitz_rolling_portfolio(
    prices_trade_period: pd.DataFrame,
    prices_full_history: pd.DataFrame,
    initial_capital: float,
    lookback_days: int = MARKOWITZ_LOOKBACK,
    transaction_cost: float = MARKOWITZ_TXN_COST,
    rebalance_frequency: str = MARKOWITZ_REBAL_FREQ,
):
    """
    Simuliert ein Markowitz-Portfolio mit rollierendem Schätzfenster.
    
    Args:
        prices_trade_period: Preise für den Trade-Zeitraum
        prices_full_history: Alle verfügbaren historischen Preise (für Lookback)
        initial_capital: Startkapital
        lookback_days: Anzahl Handelstage für rollierende Schätzung
        transaction_cost: Transaktionskosten als Dezimalzahl (0.001 = 0,1%)
        rebalance_frequency: Pandas-Frequenz für Rebalancing (z.B. "QS" für Quartalsanfang)
    
    Returns:
        df_portfolio_value: DataFrame mit täglichen Portfoliowerten und Returns
        df_rebalancing: DataFrame mit Rebalancing-Ereignissen und Gewichten
    """
    returns_trade = prices_trade_period.pct_change().dropna(how="all")
    if returns_trade.empty:
        raise ValueError("Keine Handelsdaten für den angegebenen Zeitraum.")
    
    # Rebalancing-Termine bestimmen (Quartalsanfänge)
    rebal_dates = (
        returns_trade.index.to_series()
        .resample(rebalance_frequency)
        .first()
        .dropna()
        .tolist()
    )
    
    # Ersten Handelstag hinzufügen, falls nicht bereits enthalten
    first_trade_day = returns_trade.index[0]
    if not rebal_dates or first_trade_day < rebal_dates[0]:
        rebal_dates.insert(0, first_trade_day)
    
    rebal_dates = [d for d in rebal_dates if d <= returns_trade.index[-1]]
    rebal_dates = sorted(set(rebal_dates))
    
    portfolio_value = initial_capital
    prev_weights = pd.Series(0.0, index=prices_trade_period.columns)
    
    value_records = []
    rebalancing_records = []
    
    for idx, rebal_date in enumerate(rebal_dates):
        # KORREKTUR 4: Look-Ahead Bias vermeiden!
        # Historisches Fenster MUSS STRIKT VOR dem Rebalancing-Tag enden
        # Am 03.01. kennen wir nur Daten bis 02.01. (Schlusskurs von gestern)
        hist_end_date = rebal_date - pd.Timedelta(days=1)
        hist_window = prices_full_history.loc[:hist_end_date, prices_trade_period.columns].tail(lookback_days)
        
       
        
        # Mean-Variance-Optimierung mit pypfopt (nur auf Daten BIS GESTERN)
        try:
            mu = expected_returns.mean_historical_return(hist_window, frequency=252)
            S = risk_models.sample_cov(hist_window, frequency=252)
            ef = EfficientFrontier(mu, S)
            ef.max_sharpe()  # Portfolio mit maximaler Sharpe Ratio
            weights_dict = ef.clean_weights()
            weights = pd.Series(weights_dict)
        except Exception as e:
            print(f"[Fehler] Optimierung fehlgeschlagen für {rebal_date.date()}: {e}")
            continue
        
        weights = weights.reindex(prices_trade_period.columns).fillna(0.0)
        
        
        # Transaktionskosten berechnen (Turnover = Summe der absoluten Gewichtsänderungen)
        turnover = (weights - prev_weights).abs().sum()
        txn_cost_value = turnover * transaction_cost * portfolio_value
        portfolio_value -= txn_cost_value
        
        prev_weights = weights.copy()
        
        rebalancing_records.append({
            "date": rebal_date,
            "transaction_cost": txn_cost_value,
            "turnover": turnover,
            "portfolio_value_after_costs": portfolio_value,
            **weights.to_dict(),
        })
        
        # Return-Periode: JETZT können wir ab rebal_date handeln (ex-ante korrekt!)
        if idx < len(rebal_dates) - 1:
            period_mask = (returns_trade.index >= rebal_date) & (returns_trade.index < rebal_dates[idx + 1])
        else:
            period_mask = returns_trade.index >= rebal_date
        
        period_rets = returns_trade.loc[period_mask]
        
        if period_rets.empty:
            continue
     
        portfolio_daily_returns = (period_rets * weights).sum(axis=1)
        
        # Berechne kumulative Portfolio-Werte durch kumulative Multiplikation
        cumulative_factors = (1 + portfolio_daily_returns).cumprod()
        portfolio_values = portfolio_value * cumulative_factors
        
        # Portfolio-Wert für nächste Iteration aktualisieren
        portfolio_value = portfolio_values.iloc[-1]
        
        # Records erstellen
        for dt, daily_ret, pv in zip(period_rets.index, portfolio_daily_returns, portfolio_values):
            value_records.append({
                "date": dt,
                "daily_return": daily_ret,
                "portfolio_value": pv,
            })
    
    df_portfolio = pd.DataFrame(value_records)
    df_rebal = pd.DataFrame(rebalancing_records)
    
    if not df_portfolio.empty:
        df_portfolio["date"] = pd.to_datetime(df_portfolio["date"])
        df_portfolio.sort_values("date", inplace=True)
    
    if not df_rebal.empty:
        df_rebal["date"] = pd.to_datetime(df_rebal["date"])
        df_rebal.sort_values("date", inplace=True)
    
    return df_portfolio, df_rebal


# Markowitz-Strategie ausführen
df_markowitz_portfolio, df_markowitz_rebalancing = run_markowitz_rolling_portfolio(
    prices_trade_period=prices_trade,
    prices_full_history=prices_all,
    initial_capital=initial_capital,
)

# Ergebnisse ausgeben
if df_markowitz_portfolio.empty:
    print("❌ Markowitz-Strategie lieferte keine Ergebnisse.")
else:
    final_value_mw = df_markowitz_portfolio["portfolio_value"].iloc[-1]
    cum_return_mw = (final_value_mw / initial_capital) - 1
    
    markowitz_returns = df_markowitz_portfolio.set_index("date")["daily_return"].dropna()
    sharpe_mw = np.nan
    if not markowitz_returns.empty and markowitz_returns.std() != 0:
        sharpe_mw = (markowitz_returns.mean() / markowitz_returns.std()) * np.sqrt(252)
    
    print(f"📊 Markowitz Mean-Variance Portfolio (Max Sharpe) - KORRIGIERT")
    print(f"   Zeitraum: {trade_start_date} bis {trade_end_date}")
    print(f"   Finaler Portfoliowert: ${final_value_mw:,.2f}")
    print(f"   Kumulative Rendite: {cum_return_mw:.2%}")
    if not np.isnan(sharpe_mw):
        print(f"   Sharpe Ratio (annualisiert): {sharpe_mw:.3f}")
    
    print(f"\n📈 Letzte 5 Portfoliowerte:")
    print(df_markowitz_portfolio[["date", "daily_return", "portfolio_value"]].tail())
    
    print(f"\n📊 Anzahl Trading-Tage: {len(df_markowitz_portfolio)}")
    
    if not df_markowitz_rebalancing.empty:
        num_rebal = len(df_markowitz_rebalancing)
        total_txn_costs = df_markowitz_rebalancing["transaction_cost"].sum()
        print(f"\n🔄 Rebalancing-Statistiken:")
        print(f"   Anzahl Rebalancings: {num_rebal}")
        print(f"   Gesamte Transaktionskosten: ${total_txn_costs:,.2f}")
        print(f"\n   Letzte 2 Rebalancing-Events (Gewichte):")
        display_cols = ["date", "transaction_cost", "turnover", "portfolio_value_after_costs"]
        display_cols_available = [c for c in display_cols if c in df_markowitz_rebalancing.columns]
        print(df_markowitz_rebalancing[display_cols_available].tail(2))


Finales Portfoliovermögen: 1,670,736.76 USD
Kumulative Rendite seit 2022-01-01: 67.07%
Approx. Sharpe Ratio (täglich * sqrt(252)): 0.80

Letzte 5 Portfoliowerte:
          date  daily_return  portfolio_value
955 2025-10-27      0.006461     1.670917e+06
956 2025-10-28     -0.002319     1.667043e+06
957 2025-10-29     -0.002863     1.662271e+06
958 2025-10-30      0.005626     1.671622e+06
959 2025-10-31     -0.000530     1.670737e+06

Letzte 2 Rebalancings und Gewichte:
         date  transaction_cost  portfolio_value_post_costs  AAPL  AMGN  AMZN  \
14 2025-07-01       1876.264173                1.618414e+06   0.0   0.0   0.0   
15 2025-10-01       2723.437720                1.641487e+06   0.0   0.0   0.0   

    AXP       BA  CAT  CRM  ...  MRK  MSFT  NKE     NVDA   PG  SHW     TRV  \
14  0.0  0.00000  0.0  0.0  ...  0.0   0.0  0.0  0.00000  0.0  0.0  0.0232   
15  0.0  0.03593  0.0  0.0  ...  0.0   0.0  0.0  0.07927  0.0  0.0  0.0000   

    UNH   VZ      WMT  
14  0.0  0.0  0.26642 

In [ ]:
markowitz_series = convert_daily_return_to_pyfolio_ts(df_markowitz_portfolio)

# Vergleichstabelle erstellen
perf_markowitz = timeseries.perf_stats(markowitz_series)





## Equal Weight DJI


In [ ]:
# Equal-Weight Portfolio (Buy-and-Hold, kein Rebalancing)
returns_trade = prices_trade.pct_change().dropna(how="all")

# Alle Aktien gleich gewichten (1/N)
n_stocks = len(prices_trade.columns)
equal_weights = pd.Series(1.0 / n_stocks, index=prices_trade.columns)

# Portfolio Returns berechnen (gewichteter Durchschnitt der Returns)
equal_weight_returns = (returns_trade * equal_weights).sum(axis=1)

# Portfolio-Wert über Zeit
initial_value = initial_capital
equal_weight_values = initial_value * (1 + equal_weight_returns).cumprod()

# DataFrame erstellen
df_equal_weight = pd.DataFrame({
    "date": equal_weight_returns.index,
    "daily_return": equal_weight_returns.values,
    "portfolio_value": equal_weight_values.values
})

# Ergebnisse
final_value_eq = df_equal_weight["portfolio_value"].iloc[-1]
cum_return_eq = (final_value_eq / initial_capital) - 1
sharpe_eq = (equal_weight_returns.mean() / equal_weight_returns.std()) * np.sqrt(252)

print(f"📊 Equal-Weight Portfolio (Buy-and-Hold)")
print(f"   Zeitraum: {trade_start_date} bis {trade_end_date}")
print(f"   Finaler Portfoliowert: ${final_value_eq:,.2f}")
print(f"   Kumulative Rendite: {cum_return_eq:.2%}")
print(f"   Sharpe Ratio (annualisiert): {sharpe_eq:.3f}")
print(f"   Anzahl Trading-Tage: {len(df_equal_weight)}")
